In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from nltk import download
from nltk import word_tokenize

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from keras import layers


In [3]:
download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
df = pd.read_pickle('/content/drive/My Drive/dataset/yelp_all_balanced.pkl')

In [5]:
df

text  stars  years
stars                                                                         
1     4582725  Steaks taste better at Texas Roadhouse than th...      1   2020
      4481842  Food was two star. Zero star if I could for ki...      1   2016
      4300887  Disappointing that this location's milk shake ...      1   2016
      2808251  there is very bad service. don't come here. th...      1   2020
      2839627  So after 5 months later from writing this revi...      1   2016
...                                                          ...    ...    ...
5     1834880  We happened upon Palm and Pine by accident. Ca...      5   2022
      3890871  Margaritas were great and nacho pizza was deli...      5   2018
      3092769  Great tenderloins and yummy fries. Come and su...      5   2019
      351629   So happy to finally have a Thai place within 1...      5   2021
      1537059  Prices very reasonable and food is fantastic.....      5   2018

[25000 rows x 3 columns]

In [6]:
data = df['text'].map(word_tokenize).values
total_vocabulary = set(word.lower() for review in data for word in review)  # set created from nested comprehension
print('There are {} unique words in the dataset.'.format(len(total_vocabulary)))
print('There are {} unique tweets in the dataset.'.format(len(data)))


df.head()

There are 43070 unique words in the dataset.
There are 25000 unique tweets in the dataset.


text  stars  years
stars                                                                         
1     4582725  Steaks taste better at Texas Roadhouse than th...      1   2020
      4481842  Food was two star. Zero star if I could for ki...      1   2016
      4300887  Disappointing that this location's milk shake ...      1   2016
      2808251  there is very bad service. don't come here. th...      1   2020
      2839627  So after 5 months later from writing this revi...      1   2016

In [7]:
from keras.utils import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D, CuDNNLSTM
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers, optimizers
from keras.preprocessing import text, sequence

In [8]:
# set the emotion/sentiment as our target
target = df['stars']

In [9]:
target

stars         
1      4582725    1
       4481842    1
       4300887    1
       2808251    1
       2839627    1
                 ..
5      1834880    5
       3890871    5
       3092769    5
       351629     5
       1537059    5
Name: stars, Length: 25000, dtype: int64

In [10]:
# use one hot encoding since our target is categorical
y = pd.get_dummies(target).values
#y = target


In [11]:
y

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]], dtype=uint8)

In [12]:
# use keras to create a Tokenizer object
tokenizer = text.Tokenizer(num_words=40000)  # limit to the num_words most important ones
tokenizer.fit_on_texts(list(df['text']))
tokenized_texts = tokenizer.texts_to_sequences(df['text'])
X = pad_sequences(tokenized_texts, maxlen=200)

In [13]:
X

array([[   0,    0,    0, ..., 2732,    2,  668],
       [   0,    0,    0, ...,   18,  118,   47],
       [   0,    0,    0, ...,  200,   77, 8896],
       ...,
       [   0,    0,    0, ...,   13,  160,  709],
       [   0,    0,    0, ...,  155,   36, 1152],
       [   0,    0,    0, ..., 1116,    2,  111]], dtype=int32)

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# create my NN model
model = Sequential()

embedding_size = 128
model.add(Embedding(len(total_vocabulary), embedding_size))
model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
model.add(LSTM(128, return_sequences=True))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dense(5, activation='softmax'))  # use 5 because we have 5 categories
opt = optimizers.Adam(learning_rate=0.0025)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary() # check the shape

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         5512960   
                                                                 
 bidirectional (Bidirectiona  (None, None, 256)        264192    
 l)                                                              
                                                                 
 lstm (LSTM)                 (None, None, 128)         197120    
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 50)                6

In [15]:
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.1)

Epoch 1/20
282/282 [==============================] - 16s 30ms/step - loss: 1.6012 - accuracy: 0.2289 - val_loss: 1.5746 - val_accuracy: 0.2540
Epoch 2/20
282/282 [==============================] - 8s 28ms/step - loss: 1.3672 - accuracy: 0.3516 - val_loss: 1.1802 - val_accuracy: 0.4590
Epoch 3/20
282/282 [==============================] - 8s 28ms/step - loss: 1.0816 - accuracy: 0.4872 - val_loss: 1.0807 - val_accuracy: 0.4735
Epoch 4/20
282/282 [==============================] - 8s 28ms/step - loss: 0.9229 - accuracy: 0.5766 - val_loss: 1.0677 - val_accuracy: 0.5075
Epoch 5/20
282/282 [==============================] - 8s 28ms/step - loss: 0.7845 - accuracy: 0.6679 - val_loss: 1.0656 - val_accuracy: 0.5240
Epoch 6/20
282/282 [==============================] - 8s 28ms/step - loss: 0.6452 - accuracy: 0.7455 - val_loss: 0.9176 - val_accuracy: 0.5465
Epoch 7/20
282/282 [==============================] - 8s 28ms/step - loss: 0.6341 - accuracy: 0.7276 - val_loss: 0.8395 - val_accuracy: 0.577

In [16]:
from sklearn.metrics import classification_report,accuracy_score
y_pred = model.predict(X_test) # get our predictions
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)
acc = accuracy_score(y_test, y_pred)

157/157 [==============================] - 2s 9ms/step


In [18]:
print(f"accuracy: {acc}")

accuracy: 0.624


In [19]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)
print("rmse:", rmse)

rmse: 0.6772951062292475


In [20]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_pred)
print("mae:", mae)

mae: 0.4264
